In [ ]:
from mp_api.client import MPRester

with MPRester() as mpr:
    
    docs = mpr.materials.summary.search(
        chemsys=["Pb-Ti-O","Pb-Zr-O","Pb-Ti-Zr-O"],
        spacegroup_symbol="Pm-3m", #"P4mm" "R3mm"
        fields=['material_id', 'symmetry', 'structure']
    )

In [ ]:
# Modify default MP input set to enable berry phase and born charge calculation
# https://materialsproject.github.io/atomate2/user/codes/vasp.html#modifying-input-sets

from atomate2.vasp.sets.core import StaticSetGenerator
from atomate2.vasp.sets.mp import MPGGAStaticSetGenerator
from pymatgen.io.vasp.sets import MPStaticSet

from atomate2.vasp.jobs.core import StaticMaker
# from jobflow import run_locally
from jobflow_remote import submit_flow, set_run_config
from jobflow_remote.config.base import ExecutionConfig
import yaml

structure = docs[0].structure

# create a custom input generator set with a larger ENCUT
input_set_generator = MPStaticSet(
    user_incar_settings={
        "EDIFF": 1e-5,
        "LCALCPOL": True,
        "LCALCEPS": True,
        "IBRION": 5,
        "ISMEAR": 0
        # "ENCUT": 800
    }
)

# initialise the static maker to use the custom input set generator
static_maker = StaticMaker(
    input_set_generator=input_set_generator
)

# create a job using the customised maker
flow = static_maker.make(structure)

constraint = 'cpu'

# https://matgenix.github.io/jobflow-remote/user/tuning.html#submission
resources = {
    "job_name": "MPB",
    "nodes": 1, 
    "ntasks_per_node": 4,
    "cpus_per_task": 16,
    "time": "00:30:00",
    "account": "m3828",
    "constraint": constraint,
    "qos": "debug",
    "qverbatim": "#SBATCH --gpu-bind=none\n#SBATCH --gpus=4" if constraint == 'gpu' else ""
}

atomate2_config_file = f"/global/homes/c/cyrusyc/atomate2/config/atomate2-{resources['job_name']}-{resources['constraint']}.yaml"

with open(atomate2_config_file, 'w') as f:
    if resources['constraint'] == 'gpu':
        yaml.dump({
            "VASP_CMD": f"srun -n {resources['ntasks_per_node']} -c {resources['cpus_per_task']} --cpu-bind=cores --gpu-bind=none -G {resources['ntasks_per_node']} vasp_std",
            "VASP_GAMMA_CMD": f"srun -n {resources['ntasks_per_node']} -c {resources['cpus_per_task']} --cpu-bind=cores --gpu-bind=none -G {resources['ntasks_per_node']} vasp_gam"
        }, f)
    elif resources['constraint'] == 'cpu':
        yaml.dump({
            "VASP_CMD": f"srun -n {resources['ntasks_per_node']} -c {resources['cpus_per_task']} --cpu-bind=cores vasp_std",
            "VASP_GAMMA_CMD": f"srun -n {resources['ntasks_per_node']} -c {resources['cpus_per_task']} --cpu-bind=cores vasp_gam"
        }, f)

config = ExecutionConfig(
    modules=['vasp/6.4.3-gpu'],
    export=dict(
        OMP_NUM_THREADS=1,
        OMP_PLACES="threads",
        OMP_PROC_BIND="spread"
    ),
    pre_run=f"export ATOMATE2_CONFIG_FILE={atomate2_config_file}"
)

print(submit_flow(
    flow, 
    # worker="local_shell",
    # worker='batch_worker', 
    project='FE',
    exec_config=config,
    resources=resources
))

# run_locally(static_job, create_folders=True)